In [1]:
import numpy as np
import sys
import os
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir("/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/lung/All+SMI+Flat+data")

In [3]:
!ls
!pwd

1.csv			 Lung12  Lung5_Rep1  Lung5_Rep3  Lung9_Rep1  vis   vis2
cosMX_showcase_meta.csv  Lung13  Lung5_Rep2  Lung6	 Lung9_Rep2  vis1  vis3


/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/lung/All+SMI+Flat+data


In [4]:
gene_expression = pd.read_csv('./Lung5_Rep3/Lung5_Rep3-Flat_files_and_images/Lung5_Rep3_exprMat_file.csv')
cell_boundary = pd.read_csv('./Lung5_Rep3/Lung5_Rep3-Flat_files_and_images/Lung5_Rep3_metadata_file.csv')
fov_position = pd.read_csv('./Lung5_Rep3/Lung5_Rep3-Flat_files_and_images/Lung5_Rep3_fov_positions_file.csv')


In [5]:
x_coordinate = [x / 10000 for x in list(fov_position["x_global_px"])]
y_coordinate = [y / 10000 for y in list(fov_position["y_global_px"])]


In [ ]:
"""
CosMx: 
1. All FOVs are the same dimension, 5472 x 3648 pixels
2. multiply the pixel value by 0.18 um per pixel
3. FOV area: 5472 x 3648 pixels -> 984.96um x 656.64um = 646,764.134 um2 


New Benchamrk from CosMx
1. length: 5472 pixels, width: 3648 pixels
2. simulated spot: 
    length: 5472 pixels / 5 = 1094.4 pixel = 196.992 um
    width: 3648 pixels / 4 = 912 pixel = 164.16 um
    one spot area: 196.992 um * 164.16 um = 32338.2067 um2
3. In total: 20 spots / FOV

spot_fov_cellId_mapping.csv (spot_id, fov, cell_Id)

spot_gene_expression.csv (spot_id, gene1, gene2, gene3...)

"""

In [6]:
def get_spot_fov_cellId_mapping(data_result, cell_boundary_fov_11):
    new_col_val = cell_boundary_fov_11.shape[0]* [0]
    cell_boundary_fov_11.insert(loc=0, column='spot_id', value=new_col_val)

    for i in range(cell_boundary_fov_11.shape[0]):
        one_row_sample = cell_boundary_fov_11.iloc[i]

        center_x = one_row_sample["CenterX_local_px"]
        center_y = one_row_sample["CenterY_local_px"]

        # Calculate spot_id based on CenterX_local_px and CenterY_local_px values
        spot_id = 1 + 4 * int((center_x - 1e-6) / 1094.4) + int((center_y - 1e-6) / 912)

        # Use spot_id as needed in your code
        # print(spot_id)
        data_result = data_result.append({'spot_id' : spot_id, 'fov' : one_row_sample["fov"], 'cell_ID' : one_row_sample["cell_ID"]}, ignore_index = True)
    return data_result
    
data_final_result = pd.DataFrame(columns = ['spot_id', 'fov', 'cell_ID'])
fov_ids_lst = cell_boundary['fov'].unique()
print("fov_ids_lst:", fov_ids_lst)

for fov_id in tqdm(fov_ids_lst):
    # print("fov_id:", fov_id)
    cell_boundary_fov =  cell_boundary[(cell_boundary['fov']==fov_ids_lst[fov_id-1])]
    # print("cell_boundary_fov_without_spot_id:", cell_boundary_fov.shape)
    data_final_result = get_spot_fov_cellId_mapping(data_final_result, cell_boundary_fov)
spot_celld_mapping = data_final_result


fov_ids_lst: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]


  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [01:28<00:00,  2.94s/it]


In [7]:
names = ['fov'] + ['spot-id=' + str(i) for i in range(1,21)]
fov_dic = {}
for i in names:
    fov_dic[i] = 0

In [10]:
fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = [i for i in range(1,21)]


names = ['fov'] + ['spot-id=' + str(i) for i in range(1,21)]
fov_spot_cells_stats = pd.DataFrame(columns = names)


for fov_id in fov_ids_lst:
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    
    fov_dic_sample = fov_dic
    fov_dic_sample["fov"] = fov_id
    
    for i in spot_id_lst:
        spot_id_data = fov_data[(fov_data['spot_id']==i)]
        spot_id_num = "spot-id=" + str(i)
        fov_dic_sample[spot_id_num] = spot_id_data.shape[0]
    fov_spot_cells_stats = fov_spot_cells_stats.append(fov_dic_sample, ignore_index = True)



In [11]:
spot_gene_expression = ["fov", "spot_id"]
genes_name_lst = list(gene_expression.columns)[2:]
spot_gene_expression = spot_gene_expression + genes_name_lst
spot_gene_expression = pd.DataFrame(columns = spot_gene_expression)


In [12]:
def get_spot_gene_expression(spot_cell_mapping, fov_expression, spot_id):
    genes_lst = list(fov_expression.columns)[2:]
    
    cell_id_lst = spot_cell_mapping[(spot_cell_mapping['spot_id']==spot_id)]["cell_ID"].tolist()
    
    cell_gene_expression_total = len(genes_lst)*[0]
    for cell_id in cell_id_lst:
        cell_gene_expression = fov_expression[(fov_expression['cell_ID'] == cell_id)]
        cell_gene_expression = cell_gene_expression.values.tolist()[0][2:]
        cell_gene_expression_total = np.sum([cell_gene_expression_total, cell_gene_expression], axis=0).tolist()
    
    return cell_gene_expression_total
    

In [13]:
fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = [i for i in range(1,21)]

for fov_id in tqdm(fov_ids_lst):
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    fov_gene_expression = gene_expression[(gene_expression['fov'] == fov_id)]
    
    for spot_id in spot_id_lst:
        to_append = [fov_id, spot_id]
        spot_gene_express = get_spot_gene_expression(fov_data, fov_gene_expression, spot_id)
        to_append = to_append + spot_gene_express
        a_series = pd.Series(to_append, index = spot_gene_expression.columns)
        spot_gene_expression = spot_gene_expression.append(a_series, ignore_index=True)    
  

100%|██████████| 30/30 [00:40<00:00,  1.33s/it]


In [14]:
CosMx_cell_type = pd.read_csv('./cosMX_showcase_meta.csv')
sample_cell_count_dic = {}
sample_fov_count_dic = {}
for i in range(1, 9):
    sample_cell_count_dic[i] = 0
    sample_fov_count_dic[i] = 0
# print("cell:", sample_cell_count_dic)
# print("fov:", sample_fov_count_dic)

sample_3_index_lst = []

cell_id_lst = CosMx_cell_type["cell_ID"].tolist()
# print("len of cell_id_lst:", len(cell_id_lst))

fov_lst = []

index = 0
for i in range(len(cell_id_lst)):
    sample_id = int(cell_id_lst[i].split("_")[1])
    fov_id = int(cell_id_lst[i].split("_")[2])
    cell_id = int(cell_id_lst[i].split("_")[3])

    if sample_id == 3:
        sample_3_index_lst.append(index)
        
    sample_cell_count_dic[sample_id] =  sample_cell_count_dic[sample_id] + 1
    if sample_id == 3:
        if fov_id not in fov_lst:
            fov_lst.append(fov_id)
    index += 1

print(sample_cell_count_dic)
print(fov_lst)
print(len(sample_3_index_lst))


{1: 98002, 2: 105800, 3: 97809, 4: 89975, 5: 87606, 6: 139504, 7: 71304, 8: 81236}
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
97809


In [15]:
CosMx_cell_type_sample_3 = CosMx_cell_type.iloc[sample_3_index_lst]
CosMx_cell_type_sample_3.reset_index(drop=True, inplace=True)
CosMx_cell_type_sample_3

,Unnamed: 0,cell_ID,fov,Area,AspectRatio,Width,Height,Mean.CD298,Max.CD298,Mean.G,...,Panel,Diversity,totalcounts,log10totalcounts,background,remove_flagged_cells,patient,cell_type,niche,prop_tumor_in_100_neighbors
0,203803,c_3_1_3,1,2666,1.40,70,50,1770,3611,2213,...,980p,23,40,1.602060,0.008619,False,Lung5,plasmablast,plasmablast-enriched stroma,0.11
1,203804,c_3_1_5,1,2569,1.13,61,54,3913,9346,1005,...,980p,25,40,1.602060,0.008619,False,Lung5,macrophage,plasmablast-enriched stroma,0.00
2,203805,c_3_1_6,1,2678,1.20,67,56,4786,11066,666,...,980p,59,102,2.008600,0.021980,False,Lung5,B-cell,stroma,0.00
3,203806,c_3_1_7,1,3301,1.39,75,54,1747,7005,646,...,980p,33,49,1.690196,0.010559,False,Lung5,neutrophil,stroma,0.00
4,203807,c_3_1_8,1,2087,1.02,52,51,3134,6292,581,...,980p,43,60,1.778151,0.012929,False,Lung5,endothelial,lymphoid structure,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97804,301607,c_3_30_4627,30,2565,2.35,87,37,167,1074,132,...,980p,56,90,1.954243,0.019394,False,Lung5,fibroblast,stroma,0.00
97805,301608,c_3_30_4628,30,1575,1.73,64,37,701,3242,224,...,980p,38,51,1.707570,0.010990,False,Lung5,fibroblast,lymphoid structure,0.00
97806,301609,c_3_30_4629,30,1998,2.03,73,36,270,1348,129,...,980p,77,115,2.060698,0.024781,False,Lung5,fibroblast,lymphoid structure,0.00
97807,301610,c_3_30_4632,30,563,1.04,25,24,794,1459,236,...,980p,15,20,1.301030,0.004310,False,Lung5,monocyte,lymphoid structure,0.00


In [16]:

for i in CosMx_cell_type_sample_3.index:
    CosMx_cell_type_sample_3.at[i,'cell_ID']= int(CosMx_cell_type_sample_3.iloc[i]["cell_ID"].split("_")[3])


In [17]:
sample_3_dic = {}
for key in CosMx_cell_type_sample_3["cell_type"].tolist():
    if key not in sample_3_dic:
        sample_3_dic[key] = 1
    else:
        sample_3_dic[key] = sample_3_dic[key] + 1

In [18]:
cell_type_lst = list(CosMx_cell_type_sample_3['cell_type'].unique())
cell_id_lst = CosMx_cell_type_sample_3["cell_ID"].tolist()


In [19]:
def get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type, cell_type_dic):
    existig_cell_ids_with_celltype = CosMx_cell_type['cell_ID'].unique()
    
    for cell_id in one_spot_cell_lst:
        if cell_id in existig_cell_ids_with_celltype:
            one_cell_sample = CosMx_cell_type[(CosMx_cell_type['cell_ID']==cell_id)]
            cell_type = one_cell_sample["cell_type"].values[0]
            cell_type_dic[cell_type] = cell_type_dic[cell_type] + 1
        else:
            print("cell id not found in groud truth!!!")
        
    return cell_type_dic


In [20]:
fov_ids_lst = spot_celld_mapping['fov'].unique()
spot_id_lst = [i for i in range(1,21)]

column_name_lst = ['fov', 'spot_id'] + cell_type_lst
ground_truth_table = pd.DataFrame(columns = column_name_lst)

for fov_id in fov_ids_lst:
    fov_data = spot_celld_mapping[(spot_celld_mapping['fov']==fov_id)]

    for spot_id in spot_id_lst:
        sample_dic = {}
        for i in column_name_lst:
            sample_dic[i] = 0
        
        spot_id_data = fov_data[(fov_data['spot_id']==spot_id)]
        one_spot_cell_lst = list(spot_id_data['cell_ID'].unique()) # all cell ids for one specific spot
        
        CosMx_cell_type_sample_fov = CosMx_cell_type_sample_3[(CosMx_cell_type_sample_3['fov']==fov_id)]
        sample_dic = get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type_sample_fov, sample_dic)
        
        sample_dic["fov"] = fov_id
        sample_dic["spot_id"] = spot_id
        ground_truth_table = ground_truth_table.append(sample_dic, ignore_index = True)




cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud t

In [21]:
ground_truth_table.rename(columns = {'tumor 5':'tumor'}, inplace = True)
del ground_truth_table['tumor 12']
del ground_truth_table['tumor 13']
del ground_truth_table['tumor 6']
del ground_truth_table['tumor 9']

In [22]:
def get_spot_x_y_range(x_global_px, y_global_px, fov_id, fov_spot_coordinates):
    fov_length = 5472
    fov_width = 3648
    x_l = x_global_px
    y_l = y_global_px
    x_h = x_l + fov_length
    y_h = y_l + fov_width
    
    for i in range(1, 6):  # 5
        for j in range(1, 5):  # 4
            spot_id = (i - 1) * 4 + j
            x = x_l + (5472 / 10.0) * 1 + (5472 / 5.0) * (i - 1)
            y = y_h - (3648 / 8.0) * 1 - (3648 / 4.0) * (j - 1)
            fov_spot_coordinates = fov_spot_coordinates.append(
                {'fov': fov_id, 'spot_id': spot_id, 'x': x, 'y': y},
                ignore_index=True
            )

    fov_spot_coordinates['x'] = fov_spot_coordinates['x']
    fov_spot_coordinates['y'] = fov_spot_coordinates['y']

    # fov_spot_coordinates['x'] = fov_spot_coordinates['x'] * 0.18 * 1e-4
    # fov_spot_coordinates['y'] = fov_spot_coordinates['y'] * 0.18 * 1e-4

    return fov_spot_coordinates



In [23]:
fov_spot_coordinates = pd.DataFrame(columns = ['fov', 'spot_id', 'x', 'y'])
for fov_id in fov_lst:
    x_px = fov_position[fov_position['fov']==fov_id]["x_global_px"].values[0]
    y_px = fov_position[fov_position['fov']==fov_id]["y_global_px"].values[0]
    fov_spot_coordinates = get_spot_x_y_range(x_px, y_px, fov_id, fov_spot_coordinates)
fov_spot_coordinates

,fov,spot_id,x,y
0,1.0,1.0,-9675.022222,1330.888889
1,1.0,2.0,-9675.022222,418.888889
2,1.0,3.0,-9675.022222,-493.111111
3,1.0,4.0,-9675.022222,-1405.111111
4,1.0,5.0,-8580.622222,1330.888889
...,...,...,...,...
595,30.0,16.0,15497.066667,16844.888889
596,30.0,17.0,16591.466667,19580.888889
597,30.0,18.0,16591.466667,18668.888889
598,30.0,19.0,16591.466667,17756.888889


In [28]:
spot_gene_expression = spot_gene_expression.drop(columns = ["fov", "spot_id"])
ground_truth_table = ground_truth_table.drop(columns = ["fov", "spot_id"])
fov_spot_coordinates = fov_spot_coordinates.drop(columns = ["fov", "spot_id"])



import anndata as ad
st_adata = ad.AnnData(X = spot_gene_expression.values, obs = ground_truth_table, var = pd.DataFrame(index = list(spot_gene_expression.columns)), dtype=int)
st_adata.obsm["spatial"] = fov_spot_coordinates.values

spot_sums = np.sum(st_adata.X, axis=1)
mask = spot_sums > 100
filtered_data = st_adata[mask]

file_path = "/home/luqiaolin/projects/Benchmarking_paper_code/pseudo_spot_generation/cosmx_lung/Lung5_3.h5ad"
filtered_data.write_h5ad(file_path)



